In [1]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

print('Tensorflow version:', tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version: 2.6.3


In [3]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [4]:
MIXED_PRECISION = False
XLA_ACCELERATE = True

if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

Accelerated Linear Algebra enabled


In [5]:
data_dir = "../input/buildin/"
batch_size = 32
img_height = 180
img_width = 180

In [6]:
image_size = (180, 180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/buildin",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/buildin",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 8063 files belonging to 5 classes.
Using 6451 files for training.


2022-07-25 10:52:14.768973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-25 10:52:14.863128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-25 10:52:14.863834: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-25 10:52:14.865237: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 8063 files belonging to 5 classes.
Using 1612 files for validation.


In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(180, 180,3)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
with tf.device('/GPU:0'): # training executed on NVIDIA Tesla V100
    tuner.search(train_ds, epochs=50, validation_data=val_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 05m 10s]
val_accuracy: 0.9212158918380737

Best val_accuracy So Far: 0.9292804002761841
Total elapsed time: 01h 18m 50s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.0001.



In [11]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
with tf.device('/GPU:0'): # training executed on NVIDIA Tesla V100
    history = model.fit(train_ds,validation_data=val_ds, epochs=50)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
202/202 [==============================] - 38s 178ms/step - loss: 296.0723 - accuracy: 0.5838 - val_loss: 23.2607 - val_accuracy: 0.7854
Epoch 2/50
202/202 [==============================] - 35s 167ms/step - loss: 29.3442 - accuracy: 0.7534 - val_loss: 16.0613 - val_accuracy: 0.8102
Epoch 3/50
202/202 [==============================] - 35s 169ms/step - loss: 23.1281 - accuracy: 0.7976 - val_loss: 40.2641 - val_accuracy: 0.8337
Epoch 4/50
202/202 [==============================] - 35s 168ms/step - loss: 22.2253 - accuracy: 0.8267 - val_loss: 10.7824 - val_accuracy: 0.8902
Epoch 5/50
202/202 [==============================] - 36s 171ms/step - loss: 11.6845 - accuracy: 0.8884 - val_loss: 9.0929 - val_accuracy: 0.9100
Epoch 6/50
202/202 [==============================] - 35s 168ms/step - loss: 15.1050 - accuracy: 0.8737 - val_loss: 24.5020 - val_accuracy: 0.7916
Epoch 7/50
202/202 [==============================] - 36s 171ms/step - loss: 24.7726 - accuracy: 0.8530 - val_loss: 7.

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_ds,validation_data=val_ds, epochs=best_epoch)

Epoch 1/50
202/202 [==============================] - 36s 172ms/step - loss: 321.2139 - accuracy: 0.5843 - val_loss: 24.2007 - val_accuracy: 0.8077
Epoch 2/50
202/202 [==============================] - 41s 200ms/step - loss: 61.6847 - accuracy: 0.7393 - val_loss: 35.9448 - val_accuracy: 0.7971
Epoch 3/50
202/202 [==============================] - 43s 201ms/step - loss: 43.3532 - accuracy: 0.7969 - val_loss: 247.3592 - val_accuracy: 0.4913
Epoch 4/50
202/202 [==============================] - 40s 189ms/step - loss: 43.0571 - accuracy: 0.8365 - val_loss: 17.2219 - val_accuracy: 0.8914
Epoch 5/50
202/202 [==============================] - 41s 190ms/step - loss: 39.3112 - accuracy: 0.8352 - val_loss: 25.8403 - val_accuracy: 0.8573
Epoch 6/50
202/202 [==============================] - 40s 188ms/step - loss: 38.9155 - accuracy: 0.8517 - val_loss: 28.4171 - val_accuracy: 0.8530
Epoch 7/50
202/202 [==============================] - 40s 187ms/step - loss: 13.8860 - accuracy: 0.9239 - val_loss: 

In [13]:
pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 309.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.8 MB/s eta 0:00:00
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=77057 sha256=4e40b20db0ee2aef4aceab032891f39377416da3abbb859c81d4d27ebb8e267e
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.14.0
    Uninstalling wrapt-1.14.0:
      Successfully uninstalled wrapt-1.14.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstal

In [14]:
import tensorflowjs as tfjs
tfjs_target_dir = './'
tfjs.converters.save_keras_model(model, tfjs_target_dir)